# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
output_data_file = "../output_data/weather_df.csv"
# output_data_file

types_df = pd.read_csv(output_data_file)

weather_data_df = types_df.drop(columns=["Unnamed: 0"])
weather_data_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,ushuaia,90,AR,1578933632,87,-54.80,-68.30,55.40,2.62
1,mataura,0,NZ,1578933633,91,-46.19,168.86,41.00,1.01
2,longyearbyen,20,SJ,1578933634,76,78.22,15.64,-5.80,3.36
3,cape town,0,CL,1578933634,56,-36.41,-71.90,73.40,4.70
4,hilo,90,US,1578933635,51,19.73,-155.09,86.00,10.29
5,jamestown,90,US,1578933635,93,42.10,-79.24,46.00,5.82
6,punta arenas,99,PH,1578933635,88,8.99,125.34,74.88,0.36
7,rikitea,17,PF,1578933636,79,-23.12,-134.97,77.54,11.79
8,daru,59,PG,1578933636,95,-9.08,143.21,73.27,7.07
9,avarua,84,CK,1578933636,88,-21.21,-159.78,77.00,9.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
gmaps.configure(api_key=g_key)

In [11]:
locations = weather_data_df[["Lat", "Lng"]]
# locations

humidity = weather_data_df["Humidity"].astype(float)

# humidity

# figure_layout = {
#     'width' : '1000px',
#     'height':'400px',
#     'border':'1px solid black',
#     'padding':'1px',
#     'margin':'0 auto 0 auto'
# }

# fig = gmaps.figure(layout=figure_layout)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. 
# You want to be sure the weather is ideal

new_types_df = pd.DataFrame(weather_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_types_df["Max Temp"] <= 80) & (new_types_df["Max Temp"] > 70)
wind_speed = new_types_df["Wind Speed"] < 10
cloudiness = new_types_df["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
new_types_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
3,cape town,73.40,4.70,0
188,doka,71.38,8.05,0
292,yaval,72.68,7.25,0
333,birao,70.02,8.93,0
454,corral nuevo,73.40,3.36,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df = pd.DataFrame(weather_data_df, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""
# hotel_df["Coordinates"]=""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,ushuaia,AR,-54.80,-68.30,
1,mataura,NZ,-46.19,168.86,
2,longyearbyen,SJ,78.22,15.64,
3,cape town,CL,-36.41,-71.90,
4,hilo,US,19.73,-155.09,
5,jamestown,US,42.10,-79.24,
6,punta arenas,PH,8.99,125.34,
7,rikitea,PF,-23.12,-134.97,
8,daru,PG,-9.08,143.21,
9,avarua,CK,-21.21,-159.78,


In [15]:
# coordinates_lat_lng = []

# target_city = hotel_df["City"]
# # target_city

# target_type = "hotel"

# hotel_df["Coordinates"] = hotel_df["Lat"] + "," + hotel_df["Lng"]

# radius = 5000

# params = {
#     "location:" : coordinates_lat_lng,
#     "types" : target_type,
#     "radius" : radius, 
#     "key" : g_key
# }

# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# print("A HOTEL WITHIN A 5000 METER RADIUS")

# response = requests.get(base_url, params)
# # print(response.url)

# hotel_distance = response.json()
# print(json.dumps(hotel_distance, indent=4, sort_keys=True))

target_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [16]:
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# params = {
#     "location": (hotel_df["Lat"], hotel_df["Lng"]),
#     "rankby": "distance",
#     "type" : "hotel", 
#     "key" : g_key
# }

# # params
pprint(response.json(), depth=1)

{'html_attributions': [],
 'next_page_token': 'CrQCIwEAAGlRzebUX9zuJiPBddgdyWo0l0uQ7ddP0RbVCFaEoQib_cs-IO8T52VJB5bd5PG_i7Ux3QxQ57-3KEjydBdFl3Ljd7o6XMm1-VTW293PQBbS8xoErMgc_bOVOpfALIgWi_Q5jVcvINt7Znf7rLE3tbXXzt-Q-LQCCJj9BfeYDsVFA9BsTJ9ZrlwrEH-AoC3ctLRkCChfvMz3Kk6QOn8kShjEczHWKAdn2w6KxX3wsrHDxP8ZmaooirAv7bPnCFv6tAfd7lJJ15pjRixRUtd6ZO4_SxBDZUINqRPsN_3brf6owl9gDI4jmseH2ZVJG3JSipZNQhirfVCJ2TCzBIItxNLrbHfWpZ6r_X9o9XlqAc-gMRCLsZCVkdvg2ZP20NiZ_JpDQIDKMMfmSitU3k9Y3l4SEGTZhU8Lr7IMnu9MU0e9_AwaFP9M59ZAnV5ujJ3Zu1CCcmZJZnyQ',
 'results': [...],
 'status': 'OK'}


In [17]:
hotel_df.iloc[0][0]

'ushuaia'

In [18]:
#creates a list to capture hotel_names to insert into dataframe
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [19]:
hotel_name

['Los Cauquenes Resort & Spa',
 'Falls Hotel',
 'Radisson Blu Polar Hotel, Spitsbergen',
 'Hotel Molino Viejo',
 'Grand Naniloa Hotel Hilo - a DoubleTree by Hilton',
 'Chautauqua Harbor Hotel',
 nan,
 "Pension Maro'i",
 'Daru hotel',
 'The Edgewater Resort and Spa',
 'Hotel Plaza Lebu',
 'Maringe Lagoon Lodge',
 nan,
 'Abbey Beach Resort',
 'Hôtel Hiva Oa Hanakee Lodge',
 'Pudong Holiday Hotel',
 'Seagate Hotel',
 'Hilton Garden Inn Cusco',
 'Talnakh Hotel',
 nan,
 nan,
 nan,
 'Auberge De La Chanterelle',
 'Hotel Arctic',
 nan,
 'Tourist House - Sablino',
 'Harbour House Hotel',
 nan,
 'Radisson Blu Hotel, Biarritz',
 'Embassy Beach',
 'Le Monitel',
 nan,
 'Shanti Maurice Resort & Spa',
 nan,
 'Kak Doma',
 'Casa Rodo 53',
 'Casa de Fares',
 'Amita Hotel Boutique',
 nan,
 nan,
 nan,
 'Paradise First Hotel',
 'Hotel Arena Gruesa',
 'Top of the World Hotel',
 'Royal Amazonia',
 nan,
 "L'oiseau de Paradis",
 "Land's End Boutique Hotel",
 'Avasta Resort and Spa',
 'Bismillah Hotel And Lodgi

In [20]:
hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()
hotel_df.to_csv('hotel_export.csv')

In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# hotel_info
# locations

In [27]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))